In [1]:
import logging
import sys
import os
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
os.environ['NUMEXPR_MAX_THREADS'] = '20'

In [2]:
import chromadb
from llama_index import VectorStoreIndex, ServiceContext
from llama_index.vector_stores import ChromaVectorStore
db = chromadb.PersistentClient(path="/home/pedram/projects/llm-cache/chroma_db")
chroma_collection = db.get_or_create_collection("dagster-support")

INFO:chromadb.telemetry.posthog:Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.


In [13]:
chroma_collection.peek()

{'ids': ['76e4d7e5-e8d3-483e-a278-bf0ae64f86e2',
  '30db25e7-31b0-4ef8-aad4-12fd7abefeef',
  'cccd02e7-d906-4b9e-9940-a4684a22f58e',
  'aeee94db-da2c-4bad-a08f-ce3df9fce1f2',
  '5de3bf09-395b-4a4d-a562-15c057ca258a',
  'eabb9835-5019-4dca-91e1-274140dbd9de',
  'bc020d73-6ef5-47ec-89d9-7397fe80e819',
  '40d09e40-203b-4bd3-9ea4-a40074eab0e7',
  'be4abb60-7df1-4753-8e6c-414b93d651a3',
  '553642af-36cd-4b2b-8ed5-8246e7bdd29a'],
 'embeddings': [[0.14668771624565125,
   -0.3677328824996948,
   -0.38694557547569275,
   0.12423484027385712,
   0.8499736189842224,
   0.10081133991479874,
   0.1708986908197403,
   -0.20436279475688934,
   -0.5176382660865784,
   -0.5097607970237732,
   0.058178871870040894,
   -0.21342237293720245,
   -1.3752689361572266,
   0.702673614025116,
   -0.17956744134426117,
   1.2766927480697632,
   0.5957466959953308,
   -0.042297445237636566,
   0.3960573375225067,
   0.10140849649906158,
   0.37983253598213196,
   -0.023609105497598648,
   -0.5633614659309387,
   0

In [3]:
import torch
from llama_index.prompts import PromptTemplate
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
from llama_index.llms import LlamaCPP
query_wrapper_prompt = PromptTemplate("""
You are a support bot, searching for relevant information in Github Issues and Discussions. 
You will receive a question from the user, and will try to find relevant information. If you find a relevant result, include 
the type, name, and URL of the document, as well as a possible solution to their problem.

The User's Question is: {query_str}.

Answer:
""")

"""
model_url = 'https://huggingface.co/TheBloke/CodeLlama-13B-Instruct-GGUF/resolve/main/codellama-13b-instruct.Q4_K_M.gguf'
llm = LlamaCPP(
    model_url=model_url,
    model_path=None,
    temperature=0.1,
    max_new_tokens=1000,
    context_window=3900,
    generate_kwargs={},
    model_kwargs={"n_gpu_layers": -1},
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=False,
)"""



'\nmodel_url = \'https://huggingface.co/TheBloke/CodeLlama-13B-Instruct-GGUF/resolve/main/codellama-13b-instruct.Q4_K_M.gguf\'\nllm = LlamaCPP(\n    model_url=model_url,\n    model_path=None,\n    temperature=0.1,\n    max_new_tokens=1000,\n    context_window=3900,\n    generate_kwargs={},\n    model_kwargs={"n_gpu_layers": -1},\n    messages_to_prompt=messages_to_prompt,\n    completion_to_prompt=completion_to_prompt,\n    verbose=False,\n)'

In [4]:
from llama_index import VectorStoreIndex, ServiceContext, set_global_service_context
from langchain.embeddings.huggingface import HuggingFaceBgeEmbeddings
from llama_index.response.notebook_utils import *
from llama_index.llms import OpenAI
from llama_index import VectorStoreIndex, ServiceContext, set_global_service_context

llm = OpenAI(model="gpt-3.5-turbo-16k", temperature=0, max_tokens=10000)
embed_model = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-base-en")
service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)
set_global_service_context(service_context)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-base-en
Load pretrained SentenceTransformer: BAAI/bge-base-en
INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmpne35exxx
Created a temporary directory at /tmp/tmpne35exxx
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmpne35exxx/_remote_module_non_scriptable.py
Writing /tmp/tmpne35exxx/_remote_module_non_scriptable.py
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda
Use pytorch device: cuda


In [5]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(
    vector_store,
    service_context=service_context,
    similarity_top_k=5,
    max_top_k=20
)
query_engine = index.as_query_engine(service_context=service_context)

In [6]:
response = query_engine.query("""You are a support bot, searching for relevant information in Github Issues and Discussions. 
You will receive a question from the user, and will try to find relevant information. If you find a relevant result, include 
the type, name, and URL of the document, as well as a possible solution to their problem.

The User's Question is: 
Hi, how can I pass custom config to a backfill? Currently there seems to be no such option in UI
""")
display_response(response)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

**`Final Response:`** I found a relevant Github issue that discusses the ability to pass custom config to a backfill in Dagster. The issue is titled "Enable supplying config when launching a backfill" and is currently open. You can find more information about it [here](https://github.com/dagster-io/dagster/issues/10079). 

Based on the discussion in the issue, it seems that there is currently no option in the UI to pass custom config to a backfill. However, one possible solution mentioned in the discussion is to override the run config of each partition manually. For example, instead of using the default config, you can specify custom config for each partition.

In [11]:
response = query_engine.query("""You are a support bot, searching for relevant information in Github Issues and Discussions. 
You will receive a question from the user, and will try to find relevant information. If you find a relevant result, include 
the type, name, and URL of the document, as well as a possible solution to their problem.

The User's Question is: 
Hello :wave:
General question:
I have an assets file with three basic assets defined with @asset decorator, and a basic @op and @job setup from the example in the docs.  I was curious why the @op function do_something doesn't show up in the Dagit UI?  Do they have to be attached to a graph or something logically simple?  When I run the launchpad and try to configure concurrency, the @op methods don't appear in the dynamic config UI so I'm assuming there must be a reason it doesn't show up?  There are no errors.
TIA!""")
display_response(response)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

**`Final Response:`** None

In [15]:
from llama_index.prompts import PromptTemplate

template = (
    "We have provided Github Issues and Discussions as context below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
     """
You are a support bot, searching for relevant information in Github Issues and Discussions.
You will receive a question from the user, and will try to find relevant information.
If you find a relevant result, include the type, name, and URL of the document, as well as a possible solution to their problem.
An Example Answer:

Summary: (summarize the question)
Relevant Issues: (link to the issues or discussions)
(Your Answer below)
   
Given this information, please answer the question: {query_str}\n"""
)
qa_template = PromptTemplate(template)
query_engine = index.as_query_engine(service_context=service_context, text_qa_template=qa_template)

In [16]:
question = '''Hello :wave:
General question:
I have an assets file with three basic assets defined with
@asset decorator, and a basic @op and @job setup from the example in 
the docs.  I was curious why the @op function do_something doesn't show 
up in the Dagit UI?  Do they have to be attached to a graph or something 
logically simple?  When I run the launchpad and try to configure 
concurrency, the @op methods don't appear in the dynamic config UI so
I'm assuming there must be a reason it doesn't show up?  There are no errors.
TIA!'''

response = query_engine.query(question)
display_response(response)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

**`Final Response:`** Summary: The user is asking why the @op function do_something does not show up in the Dagit UI and why it does not appear in the dynamic config UI when trying to configure concurrency.

Relevant Issues:
- Title: Show asset / job / op code in Dagit
  Type: Issue
  URL: [https://github.com/dagster-io/dagster/issues/8939](https://github.com/dagster-io/dagster/issues/8939)

(Your Answer below)
Based on the information provided, it seems that the @op function do_something is not showing up in the Dagit UI and the dynamic config UI because it is not attached to a graph or a job. In order for an @op function to be visible in the Dagit UI and the dynamic config UI, it needs to be part of a graph or a job.

To resolve this issue, you can try attaching the @op function to a graph or a job. This can be done by defining a graph using the `@graph` decorator and including the @op function as a step in the graph. Alternatively, you can define a job using the `@job` decorator and include the @op function as a solid in the job.

Once the @op function is attached to a graph or a job, it should be visible in the Dagit UI and the dynamic config UI, allowing you to configure concurrency and see the function's code.

Please refer to the following issue for more information: [Show asset / job / op code in Dagit](https://github.com/dagster-io/dagster/issues/8939)

In [17]:
def ask_question(q):
    response = query_engine.query(q)
    display_response(response)
    
ask_question(''' Hey friends, new to Dagster, and am trying to track down any good articles/documents about how best to architect a pipeline 
             — for example, I have an asset that generates a pandas DataFrame, then the next asset saves it to a database. But would it 
             be better to have an IO manager do that? But then feels like it isn’t as clear or explicit . . . so maybe an op?''')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

**`Final Response:`** Summary: The user is asking for information on how to architect a pipeline in Dagster, specifically regarding the use of assets, IO managers, and ops.

Relevant Discussions:
- Title: How do I do dynamic fan out with Dagster assets?
  - Category: Assets
  - URL: [https://github.com/dagster-io/dagster/discussions/15521](https://github.com/dagster-io/dagster/discussions/15521)
  - Possible Solution: The discussion provides an example of achieving dynamic fan out in an asset using graph-backed assets and dynamic outputs. The code snippet demonstrates how to split a data frame into chunks, process each chunk in parallel, and return the results as a new asset.

Relevant Discussions:
- Title: How can I avoid persisting large Spark dataframes between ops?
  - Category: How-to examples
  - URL: [https://github.com/dagster-io/dagster/discussions/6899](https://github.com/dagster-io/dagster/discussions/6899)
  - Possible Solution: The discussion suggests using the `mem_io_manager` to store intermediate values in memory instead of on disk. This can be achieved by configuring the `io_manager` resource to use `mem_io_manager` and using the `in_process_executor`. Additionally, it mentions the option of using vanilla Python functions instead of splitting logic across multiple ops if checkpointing and visual inspection of the DAG structure are not necessary.

Answer:
When architecting a pipeline in Dagster, there are different approaches to consider depending on your specific requirements. 

If you need to achieve dynamic fan out with assets, you can refer to the discussion titled "How do I do dynamic fan out with Dagster assets?" [here](https://github.com/dagster-io/dagster/discussions/15521). It provides an example of using graph-backed assets and dynamic outputs to split a data frame into chunks, process them in parallel, and return the results as a new asset.

On the other hand, if you want to avoid persisting large Spark dataframes between ops, you can explore the discussion titled "How can I avoid persisting large Spark dataframes between ops?" [here](https://github.com/dagster-io/dagster/discussions/6899). It suggests using the `mem_io_manager` to store intermediate values in memory instead of on disk. This can be achieved by configuring the `io_manager` resource to use `mem_io_manager` and using the `in_process_executor`. Alternatively, if checkpointing and visual inspection of the DAG structure are not necessary, you can consider using vanilla Python functions instead of splitting logic across multiple ops.

Please let me know if you need further assistance!

In [16]:
ask_question('''
Hi everyone,probably a newbie question: how can I iterate through values in an op 
so it executes downstream ops for each value?  In my case I want to migrate data 
and I need to iterate through schemas names. In the original script it's done 
simply using for loops, but I feel a Dagster job needs another approach.
''')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit

llama_print_timings:        load time =   346.00 ms
llama_print_timings:      sample time =   147.71 ms /   502 runs   (    0.29 ms per token,  3398.48 tokens per second)
llama_print_timings: prompt eval time =   614.94 ms /   874 tokens (    0.70 ms per token,  1421.28 tokens per second)
llama_print_timings:        eval time =  9451.34 ms /   501 runs   (   18.86 ms per token,    53.01 tokens per second)
llama_print_timings:       total time = 10758.96 ms


**`Final Response:`** Summary: The user wants to know how to iterate through values in an op and execute downstream ops for each value in a Dagster job.
Relevant Issues:

* <https://github.com/dagster-io/dagster/discussions/14546>
* <https://github.com/dagster-io/dagster/discussions/12016>

Possible Solution:
You can use dynamic outputs to achieve this. In your case, you can define a dynamic output that yields each schema name as an output, and then map over these outputs using the downstream op. This will allow each mapped instance of the downstream op to run in parallel.

Here's an example:
```python
@op()
def my_first_op():
    return list_of_schema_names

@op()
def my_second_op(schema_name):
    # process schema name

@graph
def process_schemas():
    schema_list = my_first_op()
    for schema in schema_list:
        yield DynamicOutput(my_second_op, schema)
```
This will execute the `my_second_op` op for each schema name in parallel.

Alternatively, you can use a dynamic job to achieve this. A dynamic job is a job that is defined at runtime and can be executed multiple times with different inputs. You can define a dynamic job that takes a list of schema names as input and executes the downstream op for each schema name in parallel.

Here's an example:
```python
@op()
def my_first_op():
    return list_of_schema_names

@op()
def my_second_op(schema_name):
    # process schema name

@graph
def process_schemas():
    schema_list = my_first_op()
    for schema in schema_list:
        yield DynamicOutput(my_second_op, schema)

@job
def dynamic_job(schema_names):
    return process_schemas.to_job(schema_names=schema_names)
```
This will execute the `process_schemas` graph for each list of schema names in parallel.

In [17]:
ask_question('''So I have a dbt_assets decorator that is pulling in 4 dbt models. All siblings. But I can't seem to pull them together into a job since I switched to using the decorator for dagster asset creation. How would I select 4 siblings inside an AssetSelection?
''')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit

llama_print_timings:        load time =   346.00 ms
llama_print_timings:      sample time =   188.87 ms /   637 runs   (    0.30 ms per token,  3372.64 tokens per second)
llama_print_timings: prompt eval time =   705.16 ms /   961 tokens (    0.73 ms per token,  1362.82 tokens per second)
llama_print_timings:        eval time = 12445.65 ms /   636 runs   (   19.57 ms per token,    51.10 tokens per second)
llama_print_timings:       total time = 14063.18 ms


**`Final Response:`** Summary: The user wants to know how to select multiple dbt models as assets in Dagster when using the `dbt_assets` decorator.
Relevant Issues:

* <https://github.com/dagster-io/dagster/issues/15800> - Raise exception when multi assets contain asset inputs that are not specified in internal asset dependencies
* <https://github.com/dagster-io/dagster/discussions/13995> - How to specify an upstream dependency for dbt-based assets

Possible Solution:
To select multiple dbt models as assets in Dagster when using the `dbt_assets` decorator, you can use the `AssetSelection` class from the `dagster.core.asset_selection` module. The `AssetSelection` class allows you to specify a list of asset keys that should be included in the job.

Here's an example of how you could use the `AssetSelection` class to select multiple dbt models as assets:
```python
from dagster import AssetOut, Definitions, asset, multi_asset
from dagster.core.asset_selection import AssetSelection

@asset
def my_upstream_asset() -> int:
    return 1

@multi_asset(outs={"asset_one": AssetOut()}, internal_asset_deps={"asset_one": set()})
def my_multi_asset(context, my_upstream_asset) -> int:
    return my_upstream_asset + 1

@dbt_assets(manifest=dbt_manifest_path)
def jaffle_shop_dbt_assets(
    context: OpExecutionContext, dbt: DbtCliResource, upstream_asset: dict
):
    yield from dbt.cli(
        ["build", "--vars", json.dumps(upstream_asset)], context=context
    ).stream()

defs = Definitions(assets=[my_upstream_asset, my_multi_asset])

# Create an AssetSelection object to select the assets you want to include in the job
asset_selection = AssetSelection(
    asset_keys=["my_upstream_asset", "my_multi_asset"],
)

# Use the AssetSelection object when creating the Dagster job
job = define_dagster_job(
    name="my_job",
    assets=[defs],
    asset_selection=asset_selection,
)
```
In this example, we're using the `AssetSelection` class to select the `my_upstream_asset` and `my_multi_asset` assets. We then use the `define_dagster_job` function to create a Dagster job that includes these selected assets.

In [18]:
ask_question('''Is there a way to provide a default configuration to an op inside a @graph_multi_asset?''')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit

llama_print_timings:        load time =   346.00 ms
llama_print_timings:      sample time =   100.57 ms /   337 runs   (    0.30 ms per token,  3350.97 tokens per second)
llama_print_timings: prompt eval time =   838.01 ms /  1026 tokens (    0.82 ms per token,  1224.32 tokens per second)
llama_print_timings:        eval time =  6530.78 ms /   336 runs   (   19.44 ms per token,    51.45 tokens per second)
llama_print_timings:       total time =  7826.19 ms


**`Final Response:`** Based on the provided context, it appears that you are asking about how to specify config for graph-backed assets or op-backed assets.

When building graph-backed assets or op-backed assets, you can define config as you normally do on ops. The example below will create an asset that requires the config defined on MyConfig:
```python
class MyConfig(Config):
    config_str: str

@op
def config_op(context, config: MyConfig):
    context.log.info(config.config_str)
    ...

# To create an asset from an op:
my_asset = AssetsDefinition.from_op(
    config_op,
    keys_by_output_name={"result": AssetKey("my_asset")},
)
```
In Dagit, you can provide config by shift + click on the "materialize" button, which will display the config launchpad.

It is also possible to define a graph-backed asset:
```python
@graph_asset
def my_graph_asset():
    return config_op()
```
This will create an asset that requires the config defined on MyConfig.

In summary, you can specify config for graph-backed assets or op-backed assets by defining a Config class and using it as input to your op function. You can then use AssetsDefinition.from_op() to create an asset from the op, or define a graph-backed asset using @graph_asset.

In [19]:
ask_question('Is it possible to turn off auto materialize policy only for a specific asset?')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit

llama_print_timings:        load time =   346.00 ms
llama_print_timings:      sample time =   120.07 ms /   411 runs   (    0.29 ms per token,  3423.06 tokens per second)
llama_print_timings: prompt eval time =   337.85 ms /   417 tokens (    0.81 ms per token,  1234.26 tokens per second)
llama_print_timings:        eval time =  7129.34 ms /   410 runs   (   17.39 ms per token,    57.51 tokens per second)
llama_print_timings:       total time =  8014.95 ms


**`Final Response:`** Yes, it is possible to turn off the auto-materialize policy for a specific asset in Dagster. You can do this by setting the `auto_materialize` parameter of the `AssetMaterializePolicy` class to `False`.

Here's an example of how you could modify the `AssetMaterializePolicy` class to disable auto-materialization for a specific asset:
```python
from dagster import AssetMaterializePolicy, AssetKey

class CustomAssetMaterializePolicy(AssetMaterializePolicy):
    def __init__(self, asset_key: AssetKey):
        self.asset_key = asset_key

    def materialize(self, context: Any) -> None:
        if context.asset_key == self.asset_key:
            return
        super().materialize(context)
```
In this example, the `CustomAssetMaterializePolicy` class inherits from the `AssetMaterializePolicy` class and overrides the `materialize` method to check whether the asset key matches the one specified in the constructor. If it does match, the policy will not materialize the asset. Otherwise, it will call the superclass's `materialize` method to perform the default behavior.

You can then use this custom policy when creating your pipeline by passing an instance of the `CustomAssetMaterializePolicy` class as a parameter to the `AssetMaterializePolicy` argument of the `PipelineDefinition` constructor:
```python
from dagster import PipelineDefinition, AssetKey

def my_pipeline():
    # ...

my_pipeline = PipelineDefinition(
    name="my_pipeline",
    asset_materialize_policy=CustomAssetMaterializePolicy(AssetKey("my_asset")),
)
```
This will disable auto-materialization for the `my_asset` asset in your pipeline.

In [20]:
ask_question('Hi, is it possible to define retry policy/strategy on graph_asset? If yes, how?')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit

llama_print_timings:        load time =   346.00 ms
llama_print_timings:      sample time =   109.18 ms /   368 runs   (    0.30 ms per token,  3370.46 tokens per second)
llama_print_timings: prompt eval time =   572.69 ms /   735 tokens (    0.78 ms per token,  1283.42 tokens per second)
llama_print_timings:        eval time =  6666.41 ms /   367 runs   (   18.16 ms per token,    55.05 tokens per second)
llama_print_timings:       total time =  7732.27 ms


**`Final Response:`** Summary: The user wants to know if it's possible to define a retry policy or strategy for a Dagster graph.
Relevant Issues:

1. <https://github.com/dagster-io/dagster/issues/5553> - Allow retry_policy to be set on job/graph
2. <https://github.com/dagster-io/dagster/issues/10103> - Configure retry strategy at instance level

Possible Solution:
It is possible to define a retry policy or strategy for a Dagster graph by setting the `retry_policy` parameter in the `GraphDefinition.to_job()` method. This parameter takes a dictionary with the following keys:

* `max_retries`: The maximum number of retries allowed for the job.
* `retry_delay`: The delay between retries, in seconds.
* `retry_exponential_backoff`: Whether to use exponential backoff for the retry delay.

Here's an example code snippet:
```python
from dagster import GraphDefinition, job

@job(retry_policy={"max_retries": 3, "retry_delay": 60})
def my_job():
    ...

my_graph = GraphDefinition.to_job(my_job)
```
This will set the retry policy for `my_job` to retry up to three times with a delay of 60 seconds between retries. The `retry_exponential_backoff` parameter is not included in this example, but it can be added if needed.

In [21]:
ask_question('Hello everyone, does any one know how can we delete partitions from the dagster cloud ?')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit

llama_print_timings:        load time =   346.00 ms
llama_print_timings:      sample time =    73.55 ms /   251 runs   (    0.29 ms per token,  3412.88 tokens per second)
llama_print_timings: prompt eval time =   567.82 ms /   773 tokens (    0.73 ms per token,  1361.36 tokens per second)
llama_print_timings:        eval time =  4616.22 ms /   250 runs   (   18.46 ms per token,    54.16 tokens per second)
llama_print_timings:       total time =  5512.14 ms


**`Final Response:`** Summary: The user wants to know how to delete partitions from Dagster Cloud.
Relevant Issues:

* <https://github.com/dagster-io/dagster/discussions/16111>
* <https://github.com/dagster-io/dagster/discussions/16114>

Possible Solution:

To delete a partition from Dagster Cloud, you can use the `dagster-cloud` CLI tool with the `--delete` flag followed by the ID of the partition you want to delete. For example:
```bash
dagster-cloud partition delete --partition 27212b249c128c2a1d47aa64278f0c9757143422
```
This will delete the specified partition from Dagster Cloud.

Note: Before deleting a partition, make sure that there are no runs or other resources associated with it. Also, check the TTL (time to live) for the partition in your agent settings to ensure that it is set to a reasonable value.

In [22]:
ask_question('''
Hello, is there a way to load assets produced by a run using the Python API if I have a run_id,  and the AssetKey for the asset I want to materialize? This is how I’m currently materialising assets:
# Materialize some asset
result = dagster.materialize(assets=[...])

# get the run id
result.run_id
''')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit

llama_print_timings:        load time =   346.00 ms
llama_print_timings:      sample time =   123.57 ms /   413 runs   (    0.30 ms per token,  3342.21 tokens per second)
llama_print_timings: prompt eval time =   563.42 ms /   832 tokens (    0.68 ms per token,  1476.70 tokens per second)
llama_print_timings:        eval time =  7945.70 ms /   412 runs   (   19.29 ms per token,    51.85 tokens per second)
llama_print_timings:       total time =  9118.08 ms


**`Final Response:`** Yes, you can load assets produced by a run using the Python API if you have a run ID and the AssetKey for the asset you want to materialize. To do this, you can use the `DagsterInstance` object to access the Dagster database and retrieve the asset materialization events from the run.

Here's an example of how you can do this:
```python
from dagster import AssetMaterialization, DagsterEventType, DagsterInstance

# Get a DagsterInstance object
instance = DagsterInstance.get()

# Get the asset materialization events for the run
materialization_records = instance.get_records_for_run(
    run_id=run_id, of_type=DagsterEventType.ASSET_MATERIALIZATION
).records

# Filter the records to get only the ones for the asset you want to materialize
asset_materialization_record = next(
    record for record in materialization_records if record.asset_key == asset_key
)

# Get the asset materialization event from the record
asset_materialization_event = asset_materialization_record.asset_materialization

# Materialize the asset using the event
result = dagster.materialize(assets=[asset_materialization_event])
```
This code will retrieve the asset materialization events for the run with the given ID, filter them to get only the ones for the asset you want to materialize, and then use the `DagsterInstance` object to materialize the asset.

Relevant Issues:

* <https://github.com/dagster-io/dagster/issues/14613>
* <https://github.com/dagster-io/dagster/discussions/10781>

In [23]:
ask_question('Hi team! How can I do full-refresh for a dbt assets on Dagster UI? Not really sure how to use full_fresh under Config (see screenshot)')


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit

llama_print_timings:        load time =   346.00 ms
llama_print_timings:      sample time =    60.78 ms /   203 runs   (    0.30 ms per token,  3340.08 tokens per second)
llama_print_timings: prompt eval time =  1075.42 ms /  1451 tokens (    0.74 ms per token,  1349.24 tokens per second)
llama_print_timings:        eval time =  4220.55 ms /   202 runs   (   20.89 ms per token,    47.86 tokens per second)
llama_print_timings:       total time =  5576.78 ms


**`Final Response:`** Summary: The user is looking for a way to trigger a full refresh of their dbt incremental models using software-defined assets in Dagster.
Relevant Issues:

* <https://github.com/dagster-io/dagster/discussions/8873>

Possible Solution:
To trigger a full refresh of your dbt incremental models, you can pass configuration to the underlying op that backs your software-defined assets. The op is named something like run_op_XXX. For example, the op that backs my dbt software-defined assets is named run_dbt_b83ee_a92a1. You can pass this configuration in the launchpad to ensure that a full refresh occurs:
```yaml
ops:
  run_dbt_b83ee_a92a1:
    config:
      full_refresh: true
```

In [24]:
ask_question('''Did anyone successfully persist history logs to a duckdb file, say instead of sqlite?
https://docs.dagster.io/deployment/dagster-instance#sqlite-storage-default''')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit

llama_print_timings:        load time =   346.00 ms
llama_print_timings:      sample time =    88.99 ms /   293 runs   (    0.30 ms per token,  3292.65 tokens per second)
llama_print_timings: prompt eval time =   456.48 ms /   595 tokens (    0.77 ms per token,  1303.46 tokens per second)
llama_print_timings:        eval time =  5680.43 ms /   292 runs   (   19.45 ms per token,    51.40 tokens per second)
llama_print_timings:       total time =  6585.81 ms


**`Final Response:`** Based on the provided Github Issues and Discussions, it appears that there have been some reports of issues with persisting history logs to a duckdb file instead of sqlite.

One relevant issue is titled "Option to specify a database schema for run/event_log/schedule storage" and can be found at the following URL: <https://github.com/dagster-io/dagster/issues/4861>. This issue discusses providing an option to specify a database schema for run/event_log/schedule storage logs, where Dagster can create the relevant tables.

Another relevant issue is titled "sqlite event log watch problems on ephemeral instance" and can be found at the following URL: <https://github.com/dagster-io/dagster/issues/3665>. This issue discusses some reports of watching a pipeline run not working, which seems to happen when a newer user is trying Dagster out and has not yet set up DAGSTER_HOME, meaning they are on the "ephemeral" instance with a sqlite db in a tempfile.TemporaryDirectory.

It appears that there have been some issues with persisting history logs to a duckdb file instead of sqlite, but it is not clear if these issues have been resolved or if there are any known solutions.

In [25]:
ask_question('''
Hello ... How do you run a "dbt source freshness" with @dbt_assets ?
There are no models associated (to be materialized) when dbt runs a source freshness so is this even possible? I.e. something like below.
My first attempt was just putting together a function that only ran a source freshness (the manifest.json has one model in it)
@dbt_assets(
    manifest=dbt_manifest_path,
    dagster_dbt_translator=CustomDagsterDbtTranslator(),
    io_manager_key="snowflake_io_manager",
)
def my_dbt_assets(context: OpExecutionContext, dbt: DbtCliResource):
    yield from dbt.cli(["source","freshness"], context=context).stream()
Running dbt command: `dbt --no-write-json --no-use-colors source freshness --target dev --select fqn:the_models_@dbt_assets`.
''')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit

llama_print_timings:        load time =   346.00 ms
llama_print_timings:      sample time =   116.26 ms /   383 runs   (    0.30 ms per token,  3294.40 tokens per second)
llama_print_timings: prompt eval time =  1321.03 ms /  1641 tokens (    0.81 ms per token,  1242.21 tokens per second)
llama_print_timings:        eval time =  8373.53 ms /   382 runs   (   21.92 ms per token,    45.62 tokens per second)
llama_print_timings:       total time = 10269.64 ms


**`Final Response:`** Summary: The user wants to run a "dbt source freshness" with @dbt_assets.
Relevant Issues:

* <https://github.com/dagster-io/dagster/issues/11752> - Emit Outputs/AssetMaterialization events as the dbt command executes, rather than waiting for run_results.json
* <https://github.com/dagster-io/dagster/discussions/14648> - How do I load a table created by dbt using an IO manager?

Possible Solution:
It is not possible to run "dbt source freshness" with @dbt_assets as there are no models associated (to be materialized) when dbt runs a source freshness. However, you can use the `dbt.cli()` method to execute any dbt command and stream the output.

Here's an example of how you could modify your code to run "dbt source freshness" with @dbt_assets:
```python
from dagster import OpExecutionContext, DbtCliResource

@dbt_assets(
    manifest=dbt_manifest_path,
    dagster_dbt_translator=CustomDagsterDbtTranslator(),
    io_manager_key="snowflake_io_manager",
)
def my_dbt_assets(context: OpExecutionContext, dbt: DbtCliResource):
    yield from dbt.cli(["source","freshness"], context=context).stream()
```
This will execute the "dbt source freshness" command and stream the output to your function. You can then process the output as needed.

In [26]:
ask_question('Is it possible to chain together jobs in dagster?')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit

llama_print_timings:        load time =   346.00 ms
llama_print_timings:      sample time =   115.97 ms /   378 runs   (    0.31 ms per token,  3259.49 tokens per second)
llama_print_timings: prompt eval time =   519.41 ms /   730 tokens (    0.71 ms per token,  1405.43 tokens per second)
llama_print_timings:        eval time =  7247.54 ms /   377 runs   (   19.22 ms per token,    52.02 tokens per second)
llama_print_timings:       total time =  8339.37 ms


**`Final Response:`** Yes, it is possible to chain together jobs in Dagster. You can use the `execute_job` API and wrap the job using `reconstructable` or `build_reconstructable_job`.

Here's an example of how you can do this:
```python
from dagster import execute_job, reconstructable, DagsterInstance

result = execute_job(
    reconstructable(my_job_name),
    instance=DagsterInstance.get(),
)
```
This will execute the job and return the result. You can then use this result to chain together multiple jobs by passing it as an input to another job.

For example:
```python
from dagster import execute_job, reconstructable, DagsterInstance

result = execute_job(
    reconstructable(my_job_name),
    instance=DagsterInstance.get(),
)

# Use the result to chain together multiple jobs
result2 = execute_job(
    reconstructable(my_other_job_name),
    instance=DagsterInstance.get(),
    input_values={"input1": result},
)
```
This will execute `my_job_name` and then use the result to execute `my_other_job_name`. The `input_values` parameter is used to pass the result of the first job as an input to the second job.

Relevant Issues:

* <https://github.com/dagster-io/dagster/discussions/14387>
* <https://github.com/dagster-io/dagster/discussions/8467>

In [11]:
ask_question('''
Hello everyone, I am just wondering if there is a methodology/best practice
around creating parameterized assets and asset graphs? I have an a scenario where 
I'm essentially having to call an API multiple times but want to declare the returned data individually for auditing. The data pipeline is the same for each so was wondering if I have to manually create each of these pipelines or if they can be autogenerated?
asset-a1 -> asset-a1-processing -> asset-combined
asset-a2 -> asset-a2-processing -> asset-combined
asset-a3 -> asset-a3-processing -> asset-combined (ed
''')


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

**`Final Response:`** Yes, there is a methodology for creating parameterized assets and asset graphs. One approach is to use kwargs with AssetIns. This allows you to define a single asset graph and pass in different parameters for each instance of the asset. In your case, you can create assets like asset-a1, asset-a2, and asset-a3, and define a common asset graph that processes the data and combines the results into asset-combined. By using kwargs, you can pass in the specific data for each asset instance when calling the asset graph. This way, you don't have to manually create each pipeline and can generate them programmatically.

In [28]:
ask_question('Hi. Is there an option to check whether an asset run was triggered as part of a backfill?')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit

llama_print_timings:        load time =   346.00 ms
llama_print_timings:      sample time =    47.52 ms /   157 runs   (    0.30 ms per token,  3303.73 tokens per second)
llama_print_timings: prompt eval time =   437.35 ms /   550 tokens (    0.80 ms per token,  1257.56 tokens per second)
llama_print_timings:        eval time =  3069.25 ms /   156 runs   (   19.67 ms per token,    50.83 tokens per second)
llama_print_timings:       total time =  3745.35 ms


**`Final Response:`** Summary: The user wants to know if it's possible to determine if a Dagster run was triggered as part of a backfill.
Relevant Issues:

* <https://github.com/dagster-io/dagster/discussions/16189>
* <https://github.com/dagster-io/dagster/issues/11779>

Possible Solution: Runs that were triggered as part of a backfill will be assigned a backfill tag that includes the backfill ID. You can inspect the tags attribute of a DagsterRun object or use the DagsterInstance.get_run_tags method to check if it has this tag.

In [18]:
ask_question('''I have successfully integrated a dbt model into Dagster. Description and metadata are correctly pulled in by Dagster for my tables and visible both in the graph (the description) and the corresponding asset detail pages . However, source details do not seem to be populated when I click on the asset details. The model runs nicely via Dagster and sources appear in the graph.

My dummy dbt schema.yml is as follows:

version: 2

models:
  - name: test_model_1
    description: "This model contains user-level revenue data aggregated at the monthly level."
    columns:
      - name: n_of_rows
    config:
      meta:
        dagster:
          group: revenue

sources:
  - name: external
    description: "This is an external source"
    database: ext
    schema: ext
    tables:
    - name: payments
      description: "User payment data"
      meta:
        dagster:
          group: revenue
More specifically:

I do get to see "This model contains user-level revenue data aggregated at the monthly level." for my test_model_1 in the graph node and on the asset profile.
But I do not see "User payment data" either in the graph node or the asset profile.''')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

**`Final Response:`** Summary: The user has integrated a dbt model into Dagster and is experiencing an issue where the source details are not populated in the asset details page.

Relevant Issues:
- [dagster-dbt] - support a custom asset metadata function (Closed Issue): This issue discusses the need to surface additional information from the dbt manifest, such as the config and meta blocks, to increase Dagster's utility as a data catalog for dbt users. It suggests implementing a custom function to map node information to metadata values.

Possible Solution:
Based on the information provided, it seems that the source details are not being populated in the asset details page. One possible solution could be to implement the custom asset metadata function discussed in the closed issue. This function could be defined in the `load_assets_from_dbt_project` or `load_assets_from_dbt_manifest` methods and map the source information from the dbt manifest to the metadata values in Dagster. By doing so, the source details should be correctly populated in the asset details page.

You can find more information about this issue and the proposed solution in the following link:
- [dagster-dbt] - support a custom asset metadata function: [Link](https://github.com/dagster-io/dagster/issues/12727)